In [6]:
from glob import glob
from os.path import join


video_root = "/data/agent_h/datasets_chunyu/clip_videos_v3/"
video_paths = sorted(glob(join(video_root,"**/*.mp4"),recursive=True))
whisper_root = "/data/agent_h/datasets_chunyu/whisper_result_v3/"
whisper_paths = sorted(glob(join(whisper_root,"*")))
mediapipe_root = "/data/agent_h/datasets_chunyu/vsr_landmark_result_v3/"
mediapipe_root = sorted(glob(join(mediapipe_root,"*")))

# 1. form whiper and face into frames

print(video_path[0])
print(whisper_paths[0])
print(mediapipe_root[0])


/data/agent_h/datasets_chunyu/clip_videos_v3/2016-07-10/2016-07-10_0.mp4
/data/agent_h/datasets_chunyu/whisper_result_v3/2016-07-10_0_whisper.json
/data/agent_h/datasets_chunyu/vsr_landmark_result_v3/2016-07-10_0.FaceLandmarkerResultListzip


In [18]:
# clean whisper
from pprint import pprint
import json
import itertools

json_path = whisper_paths[0]
with open(json_path, 'r') as fp:
    whisper_result = json.load(fp)

def get_whisper_text_windows(whisper_result, clip_len=15, window_len=3):
    # get words list
    all_words = [x['words'] for x in whisper_result['segments']]
    flattened_words = list(itertools.chain.from_iterable(all_words))
    # pprint(flattened_words)
    print(len(flattened_words))


get_whisper_text_windows(whisper_result, clip_len=15, window_len=3)


164


In [19]:
# clean mediapipe result
import zlib
import pickle
import numpy as np
import mediapipe as mp


def get_mediapipe_windows(mediapipe_path, clip_len=15, window_len=3):
    with zlib.open(mediapipe_path, 'r') as fp:
        # load pickle
        mediapipe_result = pickle.load(fp)
    print(mediapipe_result[0])

get_mediapipe_windows(mediapipe_root[0], clip_len=15, window_len=3)
    

2024-04-23 10:31:35.094737: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-23 10:31:35.134608: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 10:31:35.784264: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


AttributeError: module 'zlib' has no attribute 'open'

In [ ]:
# media

def get_clip_windows(): # debug
    pass

def get_mouth_windows_fron_clip: # use later
    pass